# clean dataset

list->dict

In [ ]:
import json

with open('raw_alldata.json', 'r', encoding='utf-8') as input_file:
    data_list = json.load(input_file)

for data in data_list:
    if(not isinstance(data['Function_Description'], list)): continue
    function_description_dict = {}
    for item in data["Function_Description"][0]:
        function_description_dict[item] = data["Function_Description"][0][item]
    data["Function_Description"] = function_description_dict

with open('raw_alldata.json', 'w') as file:
    json.dump(data_list, file, ensure_ascii=False, indent=4)


read

In [ ]:
import json, re
with open('raw_alldata.json', 'r', encoding='utf-8') as input_file:
    data_list = json.load(input_file)


delete apikey

In [ ]:
def contain_apikey(instance: dict) -> bool:
    for ans in instance['Answer']:
        if 'apikey' in ans['Action_Input'].lower(): return True
        if 'api_key' in ans['Action_Input'].lower(): return True
    return False


delete Function_Description

In [ ]:
# test data
Function_Description = {
    "CountryCountryInfo": "Get country info for the given country\nParameters: {\"countryCode\": \"Required. string. Two-character represented country code. For instance, CN or cn represents China.\"}\nOutput: Success.\n - Format: text/plain\n - Structure: #CountryInfoDto",
    "CountryAvailableCountries": "Get all available countries\nParameters: {}\nOutput: Success.\n - Format: text/plain\n - Structure: Array[#CountryV3Dto]",
    "LongWeekendLongWeekend": "Get long weekends for a given country\nParameters: {\"year\": \"Required. integer. \", \"countryCode\": \"Required. string. Two-character represented country code. For instance, CN or cn represents China.\"}\nOutput: Success.\n - Format: text/plain\n - Structure: Array[#LongWeekendV3Dto]",
    "PublicHolidayPublicHolidaysV3": "Get public holidays\nParameters: {\"year\": \"Required. integer. \", \"countryCode\": \"Required. string. Two-character represented country code. For instance, CN or cn represents China.\"}\nOutput: Public holidays.\n - Format: text/plain\n - Structure: Array[#PublicHolidayV3Dto]",
    "PublicHolidayIsTodayPublicHoliday": "Is today a public holiday\nParameters: {\"countryCode\": \"Required. string. Two-character represented country code. For instance, CN or cn represents China.\", \"countyCode\": \"string. \", \"offset\": \"integer. utc timezone offset.\"}\nOutput: Today is a public holiday.\n - Format: \n - Structure:",
    "PublicHolidayNextPublicHolidays": "Returns the upcoming public holidays for the next 365 days for the given country\nParameters: {\"countryCode\": \"Required. string. Two-character represented country code. For instance, CN or cn represents China.\"}\nOutput: Success.\n - Format: text/plain\n - Structure: Array[#PublicHolidayV3Dto]",
    "PublicHolidayNextPublicHolidaysWorldwide": "Returns the upcoming public holidays for the next 7 days\nParameters: {}\nOutput: Success.\n - Format: text/plain\n - Structure: Array[#PublicHolidayV3Dto]",
    "VersionGetVersion": "Get version of the used Nager.Date library\nParameters: {}\nOutput: Success.\n - Format: text/plain\n - Structure: #VersionInfoDto",
    "components": "The detailed output format for the tools is outlined below:\n#CountryInfoDto: Object{commonName, officialName, countryCode, region, borders: Array[#CountryInfoDto]}\n#CountryV3Dto: Object{countryCode, name}\n#LongWeekendV3Dto: Object{startDate, endDate, dayCount, needBridgeDay}\n#PublicHolidayType: \n#PublicHolidayV3Dto: Object{date, localName, name, countryCode, fixed, global, counties: Array[string], launchYear, types: Array[#PublicHolidayType]}\n#VersionInfoDto: Object{name, version}"
}

def clean_function_description(function_description: dict) -> dict:
    new_function_description = {}
    for key, value in function_description.items():
        if key == 'components' : continue
        if 'apikey' in key.lower() or 'api_key' in key.lower() : continue
        value = value[:value.find("\nOutput:")]
        new_function_description[key] = value
    return new_function_description

print(clean_function_description(Function_Description))

run

In [ ]:
data_list = [data for data in data_list if data['Instances'] != [] ]
print("after clean empty instances, remain tools num: ", len(data_list))

new_data_list = []
delete_not_used_function_num = 0
delete_not_exist_function_num = 0

for data in data_list:
    # Function_Description
    if(len(data['Function_Description']) <= 2 or len(data['Instances']) <= 3) : continue
    data['Function_Description'] = clean_function_description(data['Function_Description'])
    function_list = [key for key, _ in data['Function_Description'].items()]

    # Instances
    # check apikey
    data['Instances'] = [instance for instance in data['Instances'] if not contain_apikey(instance)]
    # check function exist
    function_used_set : set = set([])
    new_instance_list = []
    for instance in data['Instances']:
        is_valid = True
        for ans in instance['Answer']: 
            if(ans['Action'] not in function_list): 
                delete_not_exist_function_num += 1
                is_valid = False
                break
            function_used_set.add(ans['Action'])
        if(is_valid):
            new_instance_list.append(instance)
    data['Instances'] = new_instance_list

    new_function_description = {}
    for key, value in data['Function_Description'].items():
        if(key not in function_used_set): 
            delete_not_used_function_num += 1
            continue
        new_function_description[key] = value
    data['Function_Description'] = new_function_description
    
    new_data_list.append(data)

print("delete_not_exist_function_num: ", delete_not_exist_function_num)
print("delete_not_used_function_num: ", delete_not_used_function_num)


In [ ]:
total_instance_num = 0
for data in new_data_list:
    total_instance_num += len(data['Instances'])
print(total_instance_num)

In [ ]:
with open('raw_data_3k.json', 'w') as file:
    json.dump(new_data_list, file, ensure_ascii=False, indent=4)

make sure description is str

In [ ]:
import json

with open('./raw_data_3k.json', 'r', encoding='utf8') as input_f:
    data_list = json.load(input_f)

for data in data_list:
    if type(data['Description']) == str: continue
    description_str = data['Description'][0]
    data['Description'] = description_str

print(data_list[300]['Description'])
with open('./raw_data_3k.json', 'w', encoding='utf8') as output_f:
    json.dump(data_list, output_f, ensure_ascii=False, indent=4)


## Make demos

In [ ]:
import json
import random
from copy import deepcopy
from tqdm import tqdm

# random.seed(42)

with open('raw_data_3k.json', 'r', encoding='utf-8') as input_file:
    raw_data_list = json.load(input_file)

demo_data_list = []

def get_random_demos(query: dict) -> list:
    selected_action = query['Answer'][0]['Action']
    # print("selected_action: ", selected_action)
    remain_instances = [instance for instance in instances if instance['Answer'][0]['Action'] != selected_action]
    if len(remain_instances) < 3 : return []
    if selected_action in remain_actions: remain_actions.remove(selected_action)
    demos = random.sample(remain_instances, k=3)
    demo_data_list.append({
        "Name": data['Name'], 
        "Description": data['Description'], 
        "Function_Description": data['Function_Description'],
        "Demonstration": demos,
        "Query": query
    })
    return demos

TRY_TIMES = 4

for data in tqdm(raw_data_list):
    cnt = 0
    instances: list = data.get("Instances", [])
    instances.sort(key=lambda instance: len(json.dumps(instance['Answer'])), reverse=True)
    remain_actions = set([instance['Answer'][0]['Action'] for instance in instances])
    # print("init: ", remain_actions)
    
    for i in range(len(instances)):
        if cnt == TRY_TIMES: break
        if get_random_demos(instances[i]) != []: cnt += 1


print(len(demo_data_list))


In [ ]:
with open('tool_demos.json', 'w', encoding='utf-8') as output_file:
    json.dump(demo_data_list, output_file, ensure_ascii=False, indent=4)
len(demo_data_list)

random sample 200

In [ ]:
import json

with open('./tool_demos.json', 'r', encoding='utf8') as f:
    data_list = json.load(f)

sample_data_list = random.sample(data_list, 200)

with open('./tool_demos_200.json', 'w', encoding='utf8') as output_f:
    json.dump(sample_data_list, output_f, ensure_ascii=False, indent=4)

In [ ]:
import json
import random
from copy import deepcopy
from tqdm import tqdm

with open('./tool_demos.json', 'r', encoding='utf8') as f:
    data_list = json.load(f)

random.shuffle(data_list)

with open('./tool_demos.json', 'w', encoding='utf8') as output_f:
    json.dump(data_list, output_f, ensure_ascii=False, indent=4)